In [1]:
import gzip, sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
limit = 5*10**5
data = []
files = []
with gzip.open('../train_data.gz', 'rt') as f:
    for i, line in enumerate(f):
        if i > limit-1:
            break
        if i % (limit//25) == 0:
            print(f"{i}/{limit}")
        current_data = np.array(line.strip().split('\t')[1:], dtype=float)
        if current_data[-2] <= 4.0:
            continue
        data += [current_data]
        files += [line.strip().split('\t')[0]]


0/500000
20000/500000
40000/500000
60000/500000
80000/500000
100000/500000
120000/500000
140000/500000
160000/500000
180000/500000
200000/500000
220000/500000
240000/500000
260000/500000
280000/500000
300000/500000
320000/500000
340000/500000
360000/500000
380000/500000
400000/500000
420000/500000
440000/500000
460000/500000
480000/500000


In [3]:
print(len(set(files)))
data = np.array(data)
print(data.shape)

100
(300000, 1076)


In [4]:
features, labels = data[:, :1004], data[:, -1]
print(features.shape, labels.shape)
# del data

(300000, 1004) (300000,)


In [6]:
#X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=False, random_state=0)
ratio = 0.9
X_train, X_test = features[:int(data.shape[0]*ratio)], features[int(data.shape[0]*ratio):]
y_train, y_test = labels[:int(data.shape[0]*ratio)], labels[int(data.shape[0]*ratio):]
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

(270000, 1004) (270000,) (30000, 1004) (30000,)


In [7]:
reg = linear_model.Ridge(alpha = 1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(f"MSE: {mean_squared_error(y_pred, y_test)}")
print(f"MAE: {mean_absolute_error(y_pred, y_test)}")

MSE: 0.00031860757735557854
MAE: 0.012334625236690044


/home/levai/miniconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=5.5436e-09): result may not be accurate.
  overwrite_a=True).T


In [8]:
reg = linear_model.Ridge(alpha = 0.5)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(f"MSE: {mean_squared_error(y_pred, y_test)}")
print(f"MAE: {mean_absolute_error(y_pred, y_test)}")
perfs = do_perf_test_on_batch()
np.mean(perfs)

MSE: 0.0003186172107234597
MAE: 0.012334833852946758


/home/levai/miniconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=2.77162e-09): result may not be accurate.
  overwrite_a=True).T


NameError: name 'do_perf_test_on_batch' is not defined

In [12]:
from joblib import dump, load
dump(reg, 'best_regressor_sklearn_after40.joblib')

['best_regressor_sklearn_after40.joblib']

In [13]:
dump(scaler, 'nn_activations_scaler_sklearn_after40.joblib')

['nn_activations_scaler_sklearn_after40.joblib']

In [28]:
some_indices = 100
start_index = 400
lX_test = X_test[start_index:start_index+some_indices]
ly_test = y_test[start_index:start_index+some_indices]
ly_pred = reg.predict(lX_test)
print(lX_test.shape, ly_test.shape, ly_pred.shape)

(100, 1003) (100,) (100,)


In [33]:
cut = 50 # the first 50 gets 0, the last 1

order_test = np.array(list(zip(range(100), ly_test, ly_pred, ly_test > np.percentile(ly_test, cut), ly_pred > np.percentile(ly_pred, cut))))

In [34]:
print(np.sum(order_test[:, 3].astype(int) == order_test[:, 4].astype(int)))
sorted(order_test, key = lambda x: x[1])

62


[array([19.        ,  0.96914619,  0.99995631,  0.        ,  0.        ]),
 array([63.        ,  0.9745149 ,  1.01747453,  0.        ,  1.        ]),
 array([97.        ,  0.97876751,  1.01682377,  0.        ,  1.        ]),
 array([93.        ,  0.97952473,  0.99185884,  0.        ,  0.        ]),
 array([21.        ,  0.98280615,  1.0014782 ,  0.        ,  0.        ]),
 array([51.        ,  0.9834578 ,  1.01642346,  0.        ,  1.        ]),
 array([47.        ,  0.9844749 ,  1.01066732,  0.        ,  0.        ]),
 array([49.        ,  0.98506707,  1.02645743,  0.        ,  1.        ]),
 array([77.        ,  0.98636705,  0.98869503,  0.        ,  0.        ]),
 array([32.        ,  0.98913813,  1.02412057,  0.        ,  1.        ]),
 array([23.        ,  0.98967129,  1.01671135,  0.        ,  1.        ]),
 array([48.        ,  0.98989475,  1.00793195,  0.        ,  0.        ]),
 array([85.        ,  0.99026251,  0.98469019,  0.        ,  0.        ]),
 array([39.        ,  0.9

In [84]:

#print(order_test)

64


In [9]:
def do_perf_test_on_batch():
    perfs = []
    cut = 50
    layer_width = 100
    for start_index in range(0, len(X_test), layer_width):
        lX_test = X_test[start_index:start_index+layer_width]
        ly_test = y_test[start_index:start_index+layer_width]
        ly_pred = reg.predict(lX_test)
        order_test = np.array(list(zip(range(layer_width), ly_test, ly_pred, ly_test > np.percentile(ly_test, cut), ly_pred > np.percentile(ly_pred, cut))))
        perfs += [np.sum(order_test[:, 3].astype(int) == order_test[:, 4].astype(int))]
    return(perfs)

In [10]:
perfs = do_perf_test_on_batch()

In [11]:
perfs

[60,
 64,
 70,
 62,
 62,
 66,
 60,
 68,
 62,
 64,
 68,
 66,
 68,
 70,
 66,
 66,
 64,
 66,
 60,
 66,
 64,
 56,
 68,
 64,
 58,
 70,
 60,
 64,
 62,
 68,
 70,
 68,
 64,
 64,
 76,
 60,
 64,
 60,
 66,
 64,
 58,
 74,
 66,
 72,
 74,
 58,
 64,
 60,
 64,
 68,
 58,
 66,
 68,
 64,
 66,
 64,
 62,
 64,
 68,
 66,
 70,
 64,
 68,
 70,
 70,
 66,
 58,
 60,
 66,
 60,
 62,
 54,
 68,
 66,
 70,
 70,
 68,
 70,
 70,
 64,
 64,
 58,
 68,
 64,
 68,
 68,
 64,
 70,
 70,
 74,
 68,
 64,
 68,
 76,
 80,
 52,
 70,
 66,
 72,
 80,
 64,
 64,
 58,
 66,
 66,
 70,
 66,
 66,
 68,
 80,
 64,
 72,
 62,
 78,
 74,
 58,
 60,
 70,
 68,
 70,
 70,
 70,
 68,
 72,
 70,
 76,
 68,
 62,
 66,
 62,
 68,
 68,
 78,
 62,
 74,
 72,
 72,
 66,
 60,
 64,
 66,
 76,
 66,
 64,
 66,
 70,
 74,
 62,
 70,
 68,
 58,
 74,
 68,
 74,
 64,
 60,
 68,
 68,
 64,
 72,
 66,
 72,
 74,
 74,
 70,
 62,
 70,
 68,
 62,
 68,
 72,
 74,
 76,
 72,
 70,
 70,
 70,
 62,
 70,
 70,
 58,
 54,
 68,
 70,
 68,
 62,
 60,
 52,
 70,
 70,
 66,
 56,
 66,
 76,
 74,
 56,
 62,
 72,
 68,
 66,
